In [1]:
 
function Simulate_timeseries(t,
                            starting_population_post_release, 
                            genotypes_m,     
                            genotypes_f,        
                            ic_f_vals,
                            ic_m_vals,
                            fitness_matrix_f,
                            fitness_matrix_m,
                            params;
                            fitness = "a"',
                            transgenic_genotype = false,
                            release_regime = false)
    
    
    
    #define storage for n. individuals (f zygote, m zygote, f pupae, m pupae, f adult, m adult)
    nF = Array{Float64}(undef,(size(genotypes_f)[1],4,t+1))
    nM = Array{Float64}(undef,(size(genotypes_m)[1],4,t+1))

    #store starting population from input
    nF[:,:,1]=starting_population_post_release[1]
    nM[:,:,1]=starting_population_post_release[2]
    
    #store no. adults in starting pop
    genotype_Nf_adult=nF[:,4,1]
    genotype_Nm_adult=nM[:,4,1]
    #store total number of starting males
    starting_Nm = sum(genotype_Nm_adult)
    
    #For repeat releases, store mask to define which males are released each generation
    if release_regime != false
        release_mask = [a == transgenic_genotype for a in genotypes_m[:,1]]
    end

    for gen in 1:t
        #For repeat releases, release adult males into population gen t-1 to generate generation t
        #And apply the reproductive fitness costs
        if (release_regime != false)

            genotype_Nm_adult=nM[:,3,gen]
            genotype_Nm_adult[release_mask] = genotype_Nm_adult[release_mask] .+ (starting_Nm*release_regime[gen])
            nM[:,4,gen] = genotype_Nm_adult
        end
        
        #Production of male and female zygotes
        f_gen = params["f"]
        n_eggs = genotype_Nf_adult.*f_gen
        
        n_zygotes_f = Array{Float64}(undef,length(genotypes_f_string))
        for o in 1:length(genotypes_f_string)
            n_zygotes_f[o] = sum(((ic_f_vals[:,:,o].*n_eggs)').* (genotype_Nm_adult./(sum(genotype_Nm_adult))))
        end
        n_zygotes_m = Array{Float64}(undef,length(genotypes_m_string))
        for o in 1:length(genotypes_m_string)
            n_zygotes_m[o] = sum(((ic_m_vals[:,:,o].*n_eggs)').* (genotype_Nm_adult./(sum(genotype_Nm_adult))))
        end
        nF[:,1,gen+1] = n_zygotes_f
        nM[:,1,gen+1] = n_zygotes_m
        
        #fitess costs are applied to zygotes if applicable
        if fitness == "b"
            n_larvae_F = n_zygotes_f.*fitness_matrix_f[:]
            n_larvae_M = n_zygotes_m.*fitness_matrix_m[:]
        else
            n_larvae_F = n_zygotes_f
            n_larvae_M = n_zygotes_m
        end

        #Apply density dependent costs
        n_zygotes = [n_larvae_F,n_larvae_M]
        n_pupae = apply_density_dependence(params,n_zygotes)
        nF[:,2,gen+1] = n_pupae[1]
        nM[:,2,gen+1] = n_pupae[2]

        #fitness costs are applied to pupae if applicable
        if fitness == "a"
            n_adults_F = n_pupae[1].*fitness_matrix_f[:]
            n_adults_M = n_pupae[2].*fitness_matrix_m[:]
        else
            n_adults_F = n_pupae[1]
            n_adults_M = n_pupae[2]
        end
        n_adults = [n_adults_F,n_adults_M]
        nF[:,3,gen+1] = n_adults[1]
        nM[:,3,gen+1] = n_adults[2]

        #fitness costs are applied to adults if applicable
        n_reproadults_F = n_adults[1]
        n_reproadults_M = n_adults[2]
        n_reproadults = [n_reproadults_F,n_reproadults_M]
        nF[:,4,gen+1] = n_reproadults[1]
        nM[:,4,gen+1] = n_reproadults[2]

        #Store for no. adults are updated for generation t+1
        genotype_Nf_adult= n_reproadults[1]
        genotype_Nm_adult= n_reproadults[2]
        
    end
    
    output = Dict(
        "simnF" => nF,
        "simnM" => nM
        )
    
    return(output)
end
    
function apply_density_dependence(params, 
                                n_zygotes)
    
    n_zygotes_sum = sum(sort(n_zygotes[1])) + sum(sort(n_zygotes[2]))
    juvenile_survival_prob = params["OJ"]*(params["α"]/(params["α"]+n_zygotes_sum))
    n_zygote_dd_f = n_zygotes[1] * juvenile_survival_prob
    n_zygote_dd_m = n_zygotes[2] * juvenile_survival_prob

    return(n_zygote_dd_f,n_zygote_dd_m)
end
"""
Initiates the prerelease population of adults. 
Back-calculates the number of zygotes for a given adult population and parameter values. 
Returns a Nx8 matrix, where N is the number of genotypes, of numbers of zygote, pupae, adults and repro adults for females and males.
"""
function Initiate_prerelease_population_4lifestages(params, genotypes_m, genotypes_f)

    
    Rm = (params["f"]*params["OJ"])/2
    Nf = params["α"]*(Rm-1)/params["f"]

    #Simulation has 4 life stages: Zygote, Pupae, Adult, Repro adult
    #Define each frequency initially as 0
    starting_population_f = Array{Float64}(undef,(length(genotypes_f),4))
    starting_population_m = Array{Float64}(undef,(length(genotypes_m),4))
    fill!(starting_population_f,0)
    fill!(starting_population_m,0)
    
    #Starting popualtion are all WT
    #Assume all pupae, adults and reproadults are fully fertile and there is no mortality between stages
    starting_population_f[1,2] = Nf
    starting_population_m[1,2] = Nf
    starting_population_f[1,3] = Nf
    starting_population_m[1,3] = Nf
    starting_population_f[1,4] = Nf
    starting_population_m[1,4] = Nf
    
    #Assuming the population is at equilibrium AND the population is pre-release
    #No. zygotes
    starting_population_f[:,1] =  starting_population_f[:,3] .* params["f"] /2
    starting_population_m[:,1] =  starting_population_m[:,3] .* params["f"] /2

    return(starting_population_f,starting_population_m)
end


"""
Takes in a Nx8 matrix, where N is the number of genotypes, defining the numbers of genotypes for a population pre-release and 
returns a Nx8  matrix of numbers of each genotype after release of males. 
The frequency and genotype of the release males is defined by the input.
"""
function release_transgenic_males(starting_population,transgenic_genotype,release_freq,genotypes)
    
    x=deepcopy(starting_population)
    mask = [a == transgenic_genotype for a in genotypes][:]
    if sum(mask)>1
        print("error in assigning trangenic values")
    end
    #Here we make the assumption that all initially released individuals are fully fit
    x[mask,4] .= sum(starting_population[:,4]).*release_freq
    
    return(x)
end

release_transgenic_males

In [4]:
"""
Takes in a simulation output and calculates various summary metrics from the numbers of individuals/gametes. 
Returns a dictionary containing the same output data with additional entries. 
"""
function Simulation_processing_wrapper(output,falleles,malleles,genotypes_f,genotypes_m)
    alleles_list = [unique([g[i] for g in falleles])[:] for i in 1:length(falleles[1])]
    
    nF = output["simnF"]
    nM = output["simnM"]
    t = length(nF[1,1,:])

    #Make dataframe header labels
    falleles_sym = [join(t) for t in falleles]
    malleles_sym = [join(t) for t in malleles]
    header_f = Symbol.([join(t) for t in genotypes_f])
    header_m = Symbol.([join(t) for t in genotypes_m])

    #Calculate relative no. females
    pupae = [sum(nF[:,2,i]) for i in 1:t]
    adult = [sum(nF[:,3,i]) for i in 1:t]
    reproadult = [sum(nF[:,4,i]) for i in 1:t]
    total_Nf = DataFrame(hcat(pupae,adult,reproadult),["pupae","adult","reproadult"])
    relpupae = pupae./pupae[1]
    reladult = adult./adult[1]
    relreproadult = reproadult./reproadult[1]
    relative_Nf = DataFrame(hcat(relpupae,reladult,relreproadult),["pupae","adult","reproadult"])

    #Calculate allele frequencies for zygotes
    genotypenumber_zygote_female = DataFrame(Transpose(nF[:,1,:]), header_f)
    genotypenumber_zygote_male = DataFrame(Transpose(nM[:,1,:]), header_m)
    genotype_freqzf_store = freq_from_no_df(genotypenumber_zygote_female)
    genotype_freqzm_store = freq_from_no_df(genotypenumber_zygote_male)
    genotype_freqz_store = (genotype_freqzf_store.+genotype_freqzm_store)./2;
    allele_freqzf = Find_allele_frequencies_from_genotypes_df(genotypes_f,genotype_freqzf_store,alleles_list);
    allele_freqzm = Find_allele_frequencies_from_genotypes_df(genotypes_m,genotype_freqzm_store,alleles_list);
    allele_freqz = (allele_freqzf .+ allele_freqzm) ./2

    #Calculate allele frequencies for adult
    genotypenumber_adult_female = DataFrame(Transpose(nF[:,3,:]), header_f)
    genotypenumber_adult_male = DataFrame(Transpose(nM[:,3,:]), header_m)
    genotype_freqaf_store = freq_from_no_df(genotypenumber_adult_female)
    genotype_freqam_store = freq_from_no_df(genotypenumber_adult_male)
    genotype_freqa_store = (genotype_freqaf_store.+genotype_freqam_store)./2;
    allele_freqaf = Find_allele_frequencies_from_genotypes_df(genotypes_f,genotype_freqaf_store,alleles_list);
    allele_freqam = Find_allele_frequencies_from_genotypes_df(genotypes_m,genotype_freqam_store,alleles_list);
    allele_freqa = (allele_freqaf .+ allele_freqam) ./2
    
    #Calculate allele frequencies for zygotes
    genotypenumber_reproadult_female = DataFrame(Transpose(nF[:,4,:]), header_f)
    genotypenumber_reproadult_male = DataFrame(Transpose(nM[:,4,:]), header_m)
    genotype_freqrf_store = freq_from_no_df(genotypenumber_reproadult_female)
    genotype_freqrm_store = freq_from_no_df(genotypenumber_reproadult_male)
    genotype_freqr_store = (genotype_freqrf_store.+genotype_freqrm_store)./2;
    allele_freqrf = Find_allele_frequencies_from_genotypes_df(genotypes_f,genotype_freqrf_store,alleles_list);
    allele_freqrm = Find_allele_frequencies_from_genotypes_df(genotypes_m,genotype_freqrm_store,alleles_list);
    allele_freqr = (allele_freqrf .+ allele_freqrm) ./2

    #save in a dictionary
    data_dict = Dict(

        "simnF" => nF,
        "simnM" => nM,
        "number_females" => total_Nf,
        "relative_number_females" => relative_Nf,
        "genotypenumber_zygote_female" => genotypenumber_zygote_female,
        "genotypenumber_zygote_male" => genotypenumber_zygote_male,
        "genotypefreq_zygote_female" => genotype_freqzf_store,
        "genotypefreq_zygote_male" => genotype_freqzm_store,
        "genotypefreq_zygote" => genotype_freqz_store,
        "allelefreq_zygote" => allele_freqz,
        "allelefreq_zygote_female" => allele_freqzf,
        "allelefreq_zygote_male" => allele_freqzm,
        
        "genotypenumber_adult_female" => genotypenumber_adult_female,
        "genotypenumber_adult_male" => genotypenumber_adult_male,
        "genotypefreq_adult_female" => genotype_freqaf_store,
        "genotypefreq_adult_male" => genotype_freqam_store,
        "genotypefreq_adult" => genotype_freqa_store,
        "allelefreq_adult" => allele_freqa,
        "allelefreq_adult_female" => allele_freqaf,
        "allelefreq_adult_male" => allele_freqam,
        
        "genotypenumber_reproadult_female" => genotypenumber_reproadult_female,
        "genotypenumber_reproadult_male" => genotypenumber_reproadult_male,
        "genotypefreq_reproadult_female" => genotype_freqrf_store,
        "genotypefreq_reproadult_male" => genotype_freqrm_store,
        "genotypefreq_reproadult" => genotype_freqr_store,
        "allelefreq_reproadult" => allele_freqr,
        "allelefreq_reproadult_female" => allele_freqrf,
        "allelefreq_reproadult_male" => allele_freqrm,
    );
    return(data_dict)
end


"""Calculate the row-wise frequency"""
function freq_from_no_df(genotype_N_store)
    genotype_N_store./sum(sort(Matrix(genotype_N_store),dims=2),dims=2)
end

"""
Takes in the genotype frequencies of a population at a single time point and 
calculated the allele frequency.
"""
function Find_allele_frequencies_from_genotypes(genotypes,freq_genotype,alleles_list)
    """Extract the allele frequencies from the genotype frequencies"""
    allele_freqs = Vector{Float64}()
    for allele in vcat(alleles_list...)
        allele_freq = 0
        for i in 1:length(genotypes)
            if (allele in genotypes[i][:,1]) & (allele in genotypes[i][:,2])
                allele_freq = allele_freq+freq_genotype[i]
            elseif (allele in genotypes[i][:,1]) | (allele in genotypes[i][:,2])
                allele_freq = allele_freq+(freq_genotype[i]/2)
            end
        end
       push!(allele_freqs,allele_freq)
    end
    return(allele_freqs)
end
function Find_allele_frequencies_from_genotypes_df(genotypes,genotype_freq_store,alleles_list)
    header = Symbol.(vcat(alleles_list...))
    Make_dataframe([Find_allele_frequencies_from_genotypes(genotypes,row,alleles_list) for row = eachrow(genotype_freq_store)],header)
end

"""
Takes in the genotype frequencies of a population at a single time point and 
calculated the frequency of each haploid genotype (chromosome).
"""
function Find_chromosome_frequencies_from_genotypes(freq_genotype,genotypes,alleles)
    """Extract the allele frequencies from the genotype frequencies"""
    allele_freqs = []
    for allele in alleles
        allele_freq = 0
        for i in 1:length(genotypes)
            if ((allele == genotypes[i][:,1]) | (allele == genotypes[i][:,2]))
                if genotypes[i][:,1] == genotypes[i][:,2]
                    allele_freq = allele_freq+(freq_genotype[i])
                else
                    allele_freq = allele_freq+(freq_genotype[i]/ 2)
                end
            end
        end
       append!(allele_freqs,allele_freq)
    end
    return(allele_freqs)
end
function Find_chromosome_frequencies_from_genotypes_df(genotype_freq_store,genotypes,alleles)
    header = Symbol.(alleles)
    Make_dataframe([Find_chromosome_frequencies_from_genotypes(row,genotypes,alleles) for row = eachrow(genotype_freq_store)],header)
end

function Find_correlation(genotypes,alleles,genotype_freq,alleles_list,A,B)
    AB_alleles = vcat(alleles_list...)
    
    chrom_freq = Find_chromosome_frequencies_from_genotypes(genotype_freq,genotypes,alleles)
    allele_freq = Find_allele_frequencies_from_genotypes(genotypes,genotype_freq,alleles_list)

    ab_freq = sum(chrom_freq[[A in x for x in alleles] .& [B in x for x in alleles]])
    a_freq = allele_freq[findall(x->x==A,AB_alleles)[1]]
    b_freq = allele_freq[findall(x->x==B,AB_alleles)[1]]

    if a_freq > 1.0
        a_freq = round(a_freq,digits=15)
    end
    if b_freq > 1.0
        b_freq = round(b_freq,digits=15)
    end

    if (float(b_freq) > 0.0)
        cor = (ab_freq - (a_freq*b_freq))/sqrt(float(a_freq*(1-a_freq)*b_freq*(1-b_freq)))
    else
        cor = 0.0
    end
    return(cor)
end
function Find_correlation_df(genotypes,alleles,genotype_freq_store,alleles_list,A,B)
    header = ["Correlation"]
    Make_dataframe([Find_correlation(genotypes,alleles,row,alleles_list,A,B) for row = eachrow(genotype_freq_store)],header)
end


"""Processes a matrix into a dataframe"""
function Make_dataframe(store, header)
    df = DataFrame(Transpose(hcat(store...)),header)
end


Simulation_processing_wrapper

In [1]:
function find_release_rate_required(n_releases,
                                    release_freq,
                                    threshold,
                                    pdict,
                                    inputs;
                                    release_genotype=["A1" "A2"],
                                    param_mod=false,
                                    param_label=false,
                                    t=100,
                                    increment = 10,
                                    Rm=6,
                                    accuracy=0.001,
                                    fitness = "a")

    checker = false
    lim = 1000
    
    genotypes_f,
    genotypes_m,
    maternal_gt,
    paternal_gt,
    fitness_f,
    fitness_m = inputs
    
    opt_subs_pdict = deepcopy(pdict)
    
    #mod params
    if param_mod != false
        for param in 1:length(param_mod)
            opt_subs_pdict[param_label[param]]=param_mod[param]
        end
    end
    opt_subs_list = [(param,opt_subs_pdict[param]) for param in keys(pdict)];
    
    #Fitness
    fitness_f_vals = Array{Float64}(fitness_f.subs(opt_subs_list))
    fitness_m_vals = Array{Float64}(fitness_m.subs(opt_subs_list))
    
    maternal_gt_vals = Array{Float64}(maternal_gt.subs(opt_subs_list))
    paternal_gt_vals = Array{Float64}(paternal_gt.subs(opt_subs_list))

    ic_f_vals,ic_m_vals  = make_ics(maternal_gt_vals,paternal_gt_vals);

    
    #Dynamics params
    params = Dict()
    Nf_eq = 1000.0
    params["OJ"] = 1/10
    params["f"] = (2*Rm)/params["OJ"]
    params["α"] = params["f"]*Nf_eq/(Rm-1)
    #params["fm"] = 0
    
    while checker == false
        
        release_regime = repeat([0.0],t)
        for i in 1:n_releases
            release_regime[i] = release_freq
        end

        #Set up release
        starting_population_Nf,starting_population_Nm = Initiate_prerelease_population_4lifestages(params, genotypes_m,genotypes_f)
        if release_regime == false
            starting_population_post_release_m = release_transgenic_males(starting_population_Nm,release_genotype,release_freq,genotypes_m)
        else
            starting_population_post_release_m = starting_population_Nm
        end
        starting_population_post_release = [starting_population_Nf,starting_population_post_release_m]

        output = Simulate_timeseries(t,             #number of generations
                            starting_population_post_release, #[1] = female, [2] = male
                            genotypes_m,     #list of genotypes, each entry = ["A1" "A2"; "B1" "B2"]
                            genotypes_f,     #list of genotypes, each entry = ["A1" "A2"; "B1" "B2"]) 
                            ic_f_vals,
                            ic_m_vals,
                            fitness_f_vals,
                            fitness_m_vals,
                            params,
                            release_regime = release_regime,
                            transgenic_genotype = release_genotype,
                            fitness = fitness)
        
        output = Simulation_processing_wrapper(output,falleles,malleles,genotypes_f,genotypes_m);
        if ((minimum(output["relative_number_females"].adult) > threshold) & (release_freq<lim))
            release_freq = release_freq+increment
        elseif release_freq>=lim
            checker = true
            return(release_freq)
        else
            if increment != accuracy
                release_freq = release_freq-increment
                increment = increment/10
            else
                checker = true
                return(release_freq)
            end
        end
    end
end

function find_release_rate_required_nopm(n_releases,
                                    release_freq,
                                    threshold,
                                    pdict,
                                    input_vals;
                                    release_genotype=["A1" "A2"],
                                    t=100,
                                    increment = 10,
                                    Rm=6,
                                    accuracy=0.001,
                                    fitness = "a")

    checker = false
    lim = 1000
    
    genotypes_f,
    genotypes_m,
    ic_f_vals,
    ic_m_vals,
    fitness_f_vals,
    fitness_m_vals = input_vals
    
    #Dynamics params
    params = Dict()
    Nf_eq = 1000.0
    params["OJ"] = 1/10
    params["f"] = (2*Rm)/params["OJ"]
    params["α"] = params["f"]*Nf_eq/(Rm-1)
    #params["fm"] = 0
    
    while checker == false
        
        release_regime = repeat([0.0],t)
        for i in 1:n_releases
            release_regime[i] = release_freq
        end

        #Set up release
        starting_population_Nf,starting_population_Nm = Initiate_prerelease_population_4lifestages(params, genotypes_m,genotypes_f)
        if release_regime == false
            starting_population_post_release_m = release_transgenic_males(starting_population_Nm,release_genotype,release_freq,genotypes_m)
        else
            starting_population_post_release_m = starting_population_Nm
        end
        starting_population_post_release = [starting_population_Nf,starting_population_post_release_m]

        output = Simulate_timeseries(t,             #number of generations
                            starting_population_post_release, #[1] = female, [2] = male
                            genotypes_m,     #list of genotypes, each entry = ["A1" "A2"; "B1" "B2"]
                            genotypes_f,     #list of genotypes, each entry = ["A1" "A2"; "B1" "B2"]) 
                            ic_f_vals,
                            ic_m_vals,
                            fitness_f_vals,
                            fitness_m_vals,
                            params,
                            release_regime = release_regime,
                            transgenic_genotype = release_genotype,
                            fitness = fitness)
        
        output = Simulation_processing_wrapper(output,falleles,malleles,genotypes_f,genotypes_m);
        if ((minimum(output["relative_number_females"].adult) > threshold) & (release_freq<lim))
            release_freq = release_freq+increment
        elseif release_freq>=lim
            checker = true
            return(release_freq)
        else
            if increment != accuracy
                release_freq = release_freq-increment
                increment = increment/10
            else
                checker = true
                return(release_freq)
            end
        end
    end
end

find_release_rate_required_nopm (generic function with 1 method)

In [ ]:
function Simulate_single_release_example(opt_subs_list,inputs,t,release_freq,release_genotype; release_regime = false,Rm = 6,fitness = "a")

    genotypes_f,
    genotypes_m,
    maternal_gt,
    paternal_gt,
    fitness_f,
    fitness_m = inputs
    
    #Fitness
    fitness_f_vals = Array{Float64}(fitness_f.subs(opt_subs_list))
    fitness_m_vals = Array{Float64}(fitness_m.subs(opt_subs_list))
    
    maternal_gt_vals = Array{Float64}(maternal_gt.subs(opt_subs_list))
    paternal_gt_vals = Array{Float64}(paternal_gt.subs(opt_subs_list))

    ic_f_vals,ic_m_vals  = make_ics(maternal_gt_vals,paternal_gt_vals);
    
    # Define pop dynamics params
    params = Dict()
    Nf_eq = 1000.0
    params["OJ"] = 1/10
    params["f"] = (2*Rm)/params["OJ"]
    params["α"] = params["f"]*Nf_eq/(Rm-1)
    params["fm"] = 0

    #Set up release
    starting_population_Nf,starting_population_Nm = Initiate_prerelease_population_4lifestages(params, genotypes_m,genotypes_f)
    if release_regime == false
        starting_population_post_release_m = release_transgenic_males(starting_population_Nm,release_genotype,release_freq,genotypes)
    else
        starting_population_post_release_m = starting_population_Nm
    end
    starting_population_post_release = [starting_population_Nf,starting_population_post_release_m]

    output = Simulate_timeseries(t, 
                    starting_population_post_release, #[1] = female, [2] = male
                    genotypes_m,
                    genotypes_f,
                    ic_f_vals,
                    ic_m_vals,
                    fitness_f_vals,
                    fitness_m_vals,
                    params,
                    release_regime = release_regime,
                    transgenic_genotype = release_genotype,
                    fitness = fitness)
    
    out = Simulation_processing_wrapper(output,falleles,malleles,genotypes_f,genotypes_m);
    return(out)
end

In [ ]:
"""
Sorts the allele string so order is in numerical order
"""
function sort_allele(unsorted_allele)
    n1 = parse(Int64,unsorted_allele[1][2:end])
    n2 = parse(Int64,unsorted_allele[2][2:end])
    if n1 > n2
        sorted_allele = [unsorted_allele[2,:]; unsorted_allele[1,:]]
    else
        sorted_allele = [unsorted_allele[1,:]; unsorted_allele[2,:]]
    end
    return(sorted_allele)
end

"""
Sorts the genotype string so order is in numerical order and A > B > C etc...
"""
function sort_genotype(unsorted_genotype; n=1)
    
    function assign(sorted_allele,unsorted_allele,unsorted_genotype)
        sorted_genotype = Array{String}(undef,(length(loci),2))
        if sorted_allele == unsorted_allele
            sorted_genotype = unsorted_genotype
        else
            sorted_genotype[:,1] = unsorted_genotype[:,2]
            sorted_genotype[:,2] = unsorted_genotype[:,1]
        end
        return(sorted_genotype)
    end
    
    if unsorted_genotype[n,1] != unsorted_genotype[n,2]
        sorted_genotype = assign(sort_allele(unsorted_genotype[n,:]),unsorted_genotype[n,:],unsorted_genotype)
    else
        if n < length(loci)
            sorted_genotype = sort_genotype(unsorted_genotype,n=n+1)
        else
            sorted_genotype = unsorted_genotype
        end
    end
    return(sorted_genotype)
end


function make_ics(allele_matrix_f,m_allele_matrix)

    n_alleles = Integer(length(m_allele_matrix[1,:])/2)
    m_allele_matrix_m = m_allele_matrix[:,1:n_alleles]
    m_allele_matrix_f = m_allele_matrix[:,n_alleles+1:end]
    
    #production of offspring
    ih_f = Array{Float64}(undef,length(genotypes_f),length(genotypes_m),length(genotypes_f))
    ih_m = Array{Float64}(undef,length(genotypes_f),length(genotypes_m),length(genotypes_m))
    fill!(ih_f, 0.0)
    fill!(ih_m, 0.0);

    for f in 1:length(genotypes_f) #mothers
        for m in 1:length(genotypes_m) #fathers
            for e in 1:length(falleles) #autosomal allele from egg
                for s in 1:length(falleles) #autosomal allele from egg
                    #X
                    e_allele = falleles[e]
                    s_allele = falleles[s]
                    offspring_genotype = sort_genotype(hcat(e_allele,s_allele))
                    genotypes_f
                    o = findall(x->x==offspring_genotype,genotypes_f)[1]
                    ih_f[f,m,o] = ih_f[f,m,o] + (allele_matrix_f[f,e] * m_allele_matrix_f[m,s])
                end

                for s in 1:length(malleles) #autosomal allele from egg
                    #Y
                    e_allele = falleles[e]
                    s_allele = malleles[s]
                    offspring_genotype = sort_genotype(hcat(e_allele,s_allele))
                    o = findall(x->x==offspring_genotype,genotypes_m)[1]
                    ih_m[f,m,o] = ih_m[f,m,o] + (allele_matrix_f[f,e] * m_allele_matrix_m[m,s])
                end
            end
        end
    end

    return(ih_f,ih_m)
end
